In [10]:
import numpy as np
import pandas as pd
import pickle
from sklearn import linear_model

In [2]:
def get_feature_matrix(file_path):
    df = pd.read_csv(file_path, header=0)
    #the total of 24 valueable features
    #the first 1 is not helpful 
    #however and datetime data can be used
    c = len(df.iloc[0,:])
    df_ = df.iloc[:,1:c]
    df_.iloc[:,0] = pd.to_datetime(df_.iloc[:,0])

    if len(df_.iloc[0,:])==26:
        print('loading training data...')
        res = df_.iloc[:,0:-1].values
    else:
        print('loading testing data...')
        res = df_.values
    return res

def get_output(file_path):
    df = pd.read_csv(file_path,header=0)
    res = np.array(df.iloc[:,len(df.columns)-1])
    return res

In [5]:
#function to normalize the data, using the training data
def normalizeMydata(data, training_data):
    data = np.array(data, dtype=np.float32)
    training_data = np.array(training_data, dtype=np.float32)

    meanX = np.mean(training_data, axis=0)
    stdX = np.std(training_data, axis=0)

    data = (data-meanX)/stdX
    return data

def setMydateData(data):
    npdates = [np.datetime64(x) for x in data[:,0]]
    data[:,0] = (npdates - np.min(npdates))/np.timedelta64(1,'D')
    return data

In [6]:
inputX = get_feature_matrix('../data/train.csv')
inputX = setMydateData(inputX)
inputX = np.array(inputX, dtype = np.float32)

outputY = get_output('../data/train.csv')
outputY = np.array(outputY, dtype = np.float32)

loading training data...


In [8]:
X = inputX
X_sq = np.square(X)
X = np.concatenate((X,X_sq), axis=1)
X_ = normalizeMydata(X,X)
Y = outputY

In [13]:
test_data = get_feature_matrix('../data/test_features.csv')
#manipulation of datetime feature
test_data = setMydateData(test_data)

test_data_sq = np.square(test_data)
test_data = np.concatenate((test_data, test_data_sq), axis=1)

test_X = normalizeMydata(test_data, X_)

loading testing data...


In [11]:
regr = linear_model.LinearRegression()
regr.fit(X_,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [17]:
predicted_Y = regr.predict(X_)
np.sqrt((np.sum((predicted_Y-Y)**2))/len(Y))

100.80008540357676

In [26]:
regr = linear_model.Lasso(0.001)
regr.fit(X_,Y)

Lasso(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=1000,
   normalize=False, positive=False, precompute=False, random_state=None,
   selection='cyclic', tol=0.0001, warm_start=False)

In [27]:
predicted_Y = regr.predict(X_)
np.sqrt((np.sum((predicted_Y-Y)**2))/len(Y))

100.80008852254966

In [37]:
regr = linear_model.Ridge(0.001)
regr.fit(X_,Y)

Ridge(alpha=0.001, copy_X=True, fit_intercept=True, max_iter=None,
   normalize=False, random_state=None, solver='auto', tol=0.001)

In [38]:
predicted_Y = regr.predict(X_)
np.sqrt((np.sum((predicted_Y-Y)**2))/len(Y))

100.80008540357478

In [53]:
#trying to remove the outliers
dfX = pd.read_csv('../data/train.csv', header=0)
dfY = pd.read_csv('../data/test_features.csv', header=0)
#the total of 24 valueable features
#the first 1 is not helpful 
#however and datetime data can be used
c = len(dfX.iloc[0,:])
df_X = dfX.iloc[:,1:c]
df_X.iloc[:,0] = pd.to_datetime(df_X.iloc[:,0])

In [66]:
#df_X[np.abs(df_X.iloc[:,1:-1]-df_X.iloc[:,1:-1].mean())<=(3*df_X.iloc[:,1:-1].std())]
dfX_fileter = df_X[dfX.iloc[:,-1]<dfX.iloc[:,-1].quantile(0.95)]
dfX_fileter

,date,T1,RH_1,T2,RH_2,T3,RH_3,T4,RH_4,T5,...,RH_8,T9,RH_9,T_out,Press_mm_hg,RH_out,Windspeed,Visibility,Tdewpoint,Output
0,2016-03-28 13:50:00,23.225735,39.338188,21.657720,39.122556,23.570008,38.805524,21.129284,40.298505,19.635501,...,38.633770,20.806671,43.444127,9.759125,718.884793,64.586239,11.121943,38.912500,3.746057,320
2,2016-03-18 04:00:00,22.214210,35.593890,16.871747,39.417072,21.423117,35.976082,21.982405,33.399373,18.584496,...,39.689837,19.977211,38.977754,1.167857,767.836942,93.840611,1.202110,32.308113,-0.510492,60
3,2016-03-11 23:40:00,26.835080,33.864591,22.563768,56.352917,14.500739,25.448560,27.144240,17.542413,24.195545,...,22.563378,12.752434,49.225806,9.878577,663.679626,75.661300,3.926755,37.292085,-1.706397,40
4,2016-04-27 18:10:00,21.087111,36.977721,19.337688,37.532298,21.079149,34.075563,20.175959,33.206859,18.655007,...,39.825814,18.959120,31.495169,7.967731,760.442268,62.914444,3.303863,39.073913,0.480563,180
5,2016-05-26 23:40:00,31.592783,58.763686,11.385170,20.753382,16.922670,36.458935,27.285903,57.394158,7.082157,...,27.991711,28.582963,26.270907,9.952803,1220.903288,36.103748,1.179556,29.022852,0.383699,70
6,2016-05-01 07:10:00,17.840799,69.399847,5.065920,44.162084,7.233629,10.434014,29.203983,46.349315,24.597410,...,49.228633,10.647137,38.580404,12.583560,885.547748,116.072718,0.467849,41.744507,2.979861,60
7,2016-03-24 03:10:00,22.543493,39.872417,19.590587,42.218280,21.619750,39.445021,21.219027,36.867926,19.742941,...,48.001169,19.354105,41.686757,5.290938,761.968479,90.454992,1.030142,65.915917,4.815835,50
8,2016-05-22 02:20:00,22.568664,42.915783,20.398808,21.055795,27.197270,20.680884,18.300166,29.180821,24.558444,...,29.810194,27.470896,32.612718,3.583676,661.759570,89.083250,5.813244,50.840309,2.250053,50
9,2016-04-08 15:50:00,21.473226,44.014568,20.285157,39.985537,22.879495,40.823670,21.247031,37.144122,20.705713,...,38.220131,21.056931,41.308144,12.510289,785.084212,54.454549,2.695328,37.494350,2.400372,300
10,2016-03-20 13:30:00,21.776219,36.864012,19.640295,37.439177,22.660617,38.548631,19.990382,34.770592,19.022283,...,34.976018,19.055911,36.111173,6.632753,766.525707,72.397503,3.353064,38.565719,2.586387,110


In [90]:
def makeSquareInput(mat):
    M = mat
    for i in range(0, len(mat[0,:])):
        C = mat[:,i]
        M_ = (mat.T * C).T
        M = np.append(M, M_, axis=1)
    return M

In [97]:
X_g = makeSquareInput(inputX[:,1:-1])
X_gn = normalizeMydata(X_g,X_g)


array([ 320.,  370.,   60., ...,   30.,   50.,   50.], dtype=float32)

In [98]:
regr = linear_model.LinearRegression()
regr.fit(X_gn,Y)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

In [100]:
predicted_Y = regr.predict(X_gn)
np.sqrt((np.sum((predicted_Y-Y)**2))/len(Y))

98.792470834613951